# CIAN.ru

In [2]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time

In [3]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

### Находим общую ссылку
Оставляем два гэпа - для странички и для района. Сделаем это, чтобы охватить всю Москву в равной степени.

In [4]:
url = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D={}&engine_version=2&offer_type=flat&p={}'
page = 2
district = 24
url.format(district,page)

'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=24&engine_version=2&offer_type=flat&p=2'

Попробуем выкачать несколько первых ссылок на квартиры 

In [5]:
links = []
for district in range(13, 14):
    for page in range(1, 2):
        page_url =  url.format(district,page)

        search_page = requests.get(page_url)
        search_page = search_page.content
        search_page = BeautifulSoup(search_page, 'lxml')

        flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
        flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))

        for link in flat_urls:
            if link.isdigit():
                print link

149988577
150330924
47886734
148186999
150452675
150963174
149632549
149953398
148282836
149351101
49549585
150464350
145205143
150609089
150824806
148181617
19464922
149500085
150068823
150638483
149123442
149670194
38354296
149988833
150333336
148372278
150992290
149670377


Теперь попробуем перейти на первую страничку из скаченного списка. Сразу же отдельно сохраним инфу, которая расположена в "табличке" на странице. 

In [6]:
flat_url = 'http://www.cian.ru/sale/flat/' + str(link) + '/'
flat_page = requests.get(flat_url)
flat_page = flat_page.content
flat_page = BeautifulSoup(flat_page, 'lxml')

table = flat_page.find('table', attrs = {'class':'object_descr_props'})
table = html_stripper(table)

# Пользовательские функции 
Напишем пользовательские функции для всех переменных, кроме района (его задаем в url) и номера квартиры (его достанем из ссылки)

**Цена**

In [11]:
def getPrice(flat_page):
    price = flat_page.find('div', attrs={'class':'object_descr_price'})
    price = re.split('<div>|руб|\W', str(price))
    price = "".join([i for i in price if i.isdigit()][-3:]) # Склеим цену из кусочков, разделенных на порядки
    return int(price)

**Расстояние от центра**

In [13]:
# Найдем на карте координаты нулевого километра Москвы
Center_lat = 55.755817 
Center_lon = 37.617633

def getDist(flat_page):
    coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
    coords = re.split('&amp|center=|%2C', str(coords))
    # Найдем координаты квартиры
    coords_list = []
    for item in coords:
        if item[0].isdigit():
            coords_list.append(item)
    lat = float(coords_list[0])
    lon = float(coords_list[1])
    
    # Расчитаем расстояние между точками по Пифагору и округлим до 2-ух знаков после запятой
    Dist = ( (lat - Center_lat)**2 + (lon - Center_lon)**2 )**0.5
    Dist = round(Dist*70, 2)
    return Dist

**Время в пути от метро**

In [14]:
def getMetrdist(flat_page):
    # Т.к. для некоторых особо удаленных районов (типа Новой Москвы) расстояние до метро не всегда указывается, используем try
    try:
        Metrdist = flat_page.find('span', attrs={'class': 'object_item_metro_comment'})
        Metrdist = html_stripper(Metrdist)
        Metrdist = re.split('\n                |', Metrdist)[1]
    except:
        Metrdist = None
    return Metrdist

**Пешком / на машине**

In [15]:
def getWalk(flat_page):
    try:
        Walk = flat_page.find('span', attrs={'class': 'object_item_metro_comment'})
        Walk = html_stripper(Walk)
        Walk = re.split('\n                    |\n', Walk)[4]
    except:
        Walk = None
        
    if Walk == 'пешком':
        ifWalk = 1
    elif Walk == None:
        ifWalk = Walk
    else:
        ifWalk = 0
    
    return ifWalk

**Число комнат**

In [16]:
def getRooms(flat_page):
    rooms = flat_page.find('div', attrs={'class':'object_descr_title'})
    rooms = html_stripper(rooms)
    room_number = ''
    
    for i in re.split('-|\n', rooms):
        if 'комн' in i:
            break
        else:
            room_number += i
    
    room_number = "".join(room_number.split())
    
    # Для квартир-студий обозначим число комнат равноым нулю
    if room_number == 'свободнаяпланировка':
        room_number = 0
    else:
        room_number
    return room_number

# Таблица
Для таблички пропишем отдельные функции, чтобы не производить лишних вычислений.

**Номер этажа**

In [17]:
def getFloor(table):
    Floor = re.split('Этаж|Тип дома', table)[1]
    Floor = re.split('\n                |\xc2', Floor)[1]
    return Floor

**Всего этажей**

In [18]:
def getNfloors(table):
    Nfloors = re.split('Этаж|Тип дома', table)[1]
    Nfloors = re.split('\xa0|\n', Nfloors)[4]
    return Nfloors

**Новостройка / вторичка**

In [19]:
def getNew(table):
    New = re.split('Тип дома:|,', table)[1]
    New = re.split('\n            |', New)[1]
    
    if New == 'новостройка': 
        ifnew = 1
    elif New == 'вторичка': 
        ifnew = 0
    else:
        ifnew = New # Т.к. всегда попадутся нестандартные варианты заполнения той или иной формы, 
                    # подстрахуемся от неверного присвоения 0 или 1
    
    return ifnew

**Материал здания**

In [20]:
def getBrick(table):
    Brick = re.split(',|Тип продажи', table)[1]
    Brick = re.split('\n            |\n', Brick)[1]
    
    if Brick == 'монолитно-кирпичный' or Brick == 'монолитный' or Brick == 'кирпичный' or Brick == 'кирпично-монолитный': 
        ifBrick = 1
    else:
        ifBrick = 0 
    
    return ifBrick

**Общая площадь**

In [21]:
def getTotsp(table):
    Totsp = re.split('Общая площадь:|м2', table)[1]
    Totsp = re.split('\n|\xc2', Totsp)[2]
    
    # Т.к. площадь может быть целой или дробной (написанной с запятой), то сначала проверим, разделяется ли она через запятую
    try: 
        Totsp_dot = re.split(',', Totsp)
        Totsp = float(Totsp_dot[0] + '.' + Totsp_dot[1])
    except: # если не разделяется, проверяем представлено на странице целое число или же что-то другое (пропущенное значение или текст)
        try: 
            Totsp = int(Totsp)
        except:
            Totsp = None
        
    return Totsp

**Жилая площадь**

In [22]:
def getLivesp(table):
    Livesp = re.split('Жилая площадь:', table)[1]
    Livesp = re.split('м2',Livesp)[0]
    Livesp = re.split('\n|\xc2', Livesp)[2]
    
    try: 
        Livesp_dot = re.split(',', Livesp)
        Livesp = float(Livesp_dot[0] + '.' + Livesp_dot[1])
    except:
        try: 
            Livesp = int(Livesp)
        except:
            Livesp = None
        
    return Livesp

**Площадь кухни**

In [23]:
def getKitsp(table):
    Kitsp  = re.split('Площадь кухни:', table)[1]
    Kitsp = re.split('м2',Kitsp)[0]
    Kitsp  = re.split('\n|\xc2', Kitsp)[2]
    
    try: 
        Kitsp_dot = re.split(',', Kitsp)
        Kitsp = float(Kitsp_dot[0] + '.' + Kitsp_dot[1])
    except:
        try: 
            Kitsp = int(Kitsp)
        except:
            Kitsp = None

    return Kitsp

**Наличие балкона**

In [24]:
def getBal(table):
    Bal = re.split('Балкон:|Лифт', table)[1]
    Bal = re.split('\n| ', Bal)[1]
    
    # Т.к. "0 балконов" обычно все-таки не пишут, просто проверим, число ли поставлено в данном месте 
    if Bal.isdigit():
        ifBal = 1
    else:
        ifBal = 0
    
    return ifBal

**Наличие телефона**

In [25]:
def getTel(table):
    Tel = re.split('Телефон:|Вид из окна', table)[1]
    Tel = re.split('\n| ', Tel)[1]
    
    if Tel == 'да' or Tel == 'есть':
        ifTel = 1
    else:
        ifTel = 0
    
    return ifTel

# Теперь парсим
Создадим пустой датафрейм, куда загрузим всю информацию

In [54]:
CianFlats = pd.DataFrame(columns=['N', 'Rooms', 'Price', 'Totsp', 'Livesp', 'Kitsp', 'Dist',  
                                    'Metrdist', 'Walk', 'Brick', 'Tel', 'Bal', 'Floor', 'Nfloors', 'New'])

In [ ]:
flatStats = {}
for district in range(13, 348): # Первые 12 цифр означают округа, а далее до 347 следуют районы Москвы
    for page in range(1, 4):
        page_url =  url.format(district,page)

        search_page = requests.get(page_url)
        search_page = search_page.content
        search_page = BeautifulSoup(search_page, 'lxml')

        flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
        flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))

        for link in flat_urls:
            if link.isdigit():
               
                flat_url = 'http://www.cian.ru/sale/flat/' + str(link) + '/'
                flat_page = requests.get(flat_url)
                flat_page = flat_page.content
                flat_page = BeautifulSoup(flat_page, 'lxml')

                flatStats['N'] = link
                flatStats['District'] = district
                flatStats['Price'] = getPrice(flat_page)
                flatStats['Dist'] = getDist(flat_page)
                flatStats['Metrdist'] = getMetrdist(flat_page)
                flatStats['Walk'] = getWalk(flat_page)
                flatStats['Rooms'] = getRooms(flat_page)

                table = flat_page.find('table', attrs = {'class':'object_descr_props'})
                table = html_stripper(table)

                flatStats['Floor'] = getFloor(table)
                flatStats['Nfloors'] = getNfloors(table)
                flatStats['New'] = getNew(table)
                flatStats['Brick'] = getBrick(table)
                flatStats['Totsp'] = getTotsp(table)
                flatStats['Livesp'] = getLivesp(table)
                flatStats['Kitsp'] = getKitsp(table)
                flatStats['Bal'] = getBal(table)
                flatStats['Tel'] = getTel(table)

                CianFlats = CianFlats.append(flatStats, ignore_index=True)
        print ('district {} page {} done'.format(district, page))

In [ ]:
CianFlats['District'].value_counts()

In [ ]:
CianFlats.to_csv('CianFlats13-347_4pages.csv')